<a href="https://colab.research.google.com/github/4may/MIResearch/blob/master/MI_survey_at_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIの最新技術調査

右も左もわからん状態でやり始めると、何やってるのかよくわからん・・・となる。特に、ドメイン知識がない分野だと以下のstep1とstep4がお手上げ状態。

1. 課題を決める
2. 解決方法を決める
3. 解く
4. 評価

そこで、いったんsurveyを読むことで、MIにはどんな課題があって、どんな方法が好まれてて、何を基準に評価したらいいのか、みたいなことを知る。

[Recent advances and applications of machine learning in solid-
state materials science](https://www.nature.com/articles/s41524-019-0221-0)

2019年に提出されたやつ。読むぞ。参考文献の数500はヤバイ笑

## Introduction

* 従来の材料開発では、**人の手による実験**が行われていた
  * 材料開発の実験は、**コスパが悪い**。投資した時間やお金に対して、調べられる材料の数が少ない。
  * 一部には、人の**カン**頼みなところがあった。
* PCを使う材料開発手法の登場
  * **DFT(Density Functional Theory)**
  * monte calro
  * molecular dynamics
  * これらのおかげで、材料開発はいくらかコスパが良くなった。
* PCの性能発展により、high-throughput計算が可能になった。
  * high-throughput計算とは、高度にシステム化された実験環境において、短期間に大量の化合物を評価して、新規の化合物を発見する手法のこと。大体、2,3ヶ月かけて、数万から数十万単位の評価を行う。
  * [参考](https://www.jstage.jst.go.jp/article/kagakutoseibutsu1962/38/4/38_4_264/_pdf)
* **機械学習アルゴリズム**の登場
  * 材料開発での応用事例
    * 安定な材料の予測。特に、熱電、光電材料
    * 材料特性の計算
    * 第一原理計算の高速化
    * リチウムイオン電池
    * 原子シミュレーション
* MIにおける機械学習の利用は**まだ始まったばかり**
  * 以下のようなことが原因で、機械学習の良さを引き出せていない
    * 材料データの数が少ない
    * 材料データのプロパティ数(特徴量)の数が少ない
* 機械学習アルゴリズムは、**ブラックボックス**。なぜ上手くいってるかが良くわからない。
* 特に、固体材料を中心に調べていく

## 機械学習の基本原理

* MIでは、**教師あり学習**を使うことが多い
* MIにおけるcross validation
  * leave-one-out CVは避けるべき。構造と特性の関係を評価するには不十分なため。
  * 未知の材料探索においては、k-fold CVも**楽観的すぎる評価**となるのでいまいち。
  * [**leave-one-cluster-out CV**](https://pubs.rsc.org/en/content/articlelanding/2018/me/c8me00012c#!divAbstract)は、material scienceのために開発された評価方法
    * 機械学習モデルが、未知の材料データをどれだけ推定できるかを計る指標
* leave-one-cluster-outについて
  * アルゴリズム

  1. 材料データをk個のクラスタに分割する
  2. 1個のクラスタを予測のために除外する
  3. 残りのk-1個のクラスタを使って学習する
  4. step2のクラスタを使って予測する

  step1のkの決め方は以下の通り。
  
  * kを決めるためのアルゴリズム

  1. 入力データをnormalizeする
  2. 以下、n回繰り返す.nはCVの回数
    
    * k-meansのセントロイド初期化が上手くいくように、データをシャッフルする
    * k-meansをk=2からk=10まで行う。10としたのは、k fold CVにおいて、10が上限値とされることが多いため。
    * 以下、k個に分かれているクラスタごとに以下を行う。どのkが一番良い結果を生むか調べる。

      * 任意のクラスタを一つ、データから除外する
      * 残りのk-1個のクラスタから成るデータを使って、学習する
      * 除外したクラスタを使って、予測する
      * 各k毎に結果の平均値と標準偏差をまとめる

  * なお、k-meansでいろんなkを試す代わりに、X-meansやG-meansを使って最適なkを計算しても良い。この場合、2-1 ~ 2.3.4のk-meansを使う複数の処理を一つの処理に置き換えられることになる。

### Databases

* データを公平に扱うのが大事
  * 4つの原則

  1. findability
  2. accessibility
  3. interoperability(相互運用性) : いろんな企業、機関が使える。
  4. repurposability(再利用性) : 別の目的(材料開発以外)に使える。

* メタ情報やフォーマットを揃えるのが理想
* 上手くいかなかった例も共有しよう
  * 機械学習の分野では、上手くいったパターンのみが公開されがち。
  * 失敗は成功のもと！
  * 車輪の再失敗を防ぐためにも、袋小路は見つけたら教えよう

### Features

* 材料開発における特徴量とは、**異なる原子や結晶構造を区別するための情報**である
* **記述子(descriptors)** : 化学式を数値に変換したもの。この数値は材料特性を踏まえて作成される。(ただ、1から順に番号を振るだけではない)。数値は、数十〜数百の数字で構成される。
  * 記述子には相関がない方が良い。大量の相関があると、次元の呪いに陥る可能性がある。
  * descriptorの計算方法には、いくつか種類がある。
  * pythonだと、[matminer](https://hackingmaterials.lbl.gov/matminer/#data-descriptor-tools)や[xenonpy](https://ma.issp.u-tokyo.ac.jp/app/1434)を使って計算できる。xenonpyのdescriptor計算は、基本的にmatminerのラッパーとなっている。
  * 基本的には、原子の動きから計算する。原子の動きとは、rotation、reflection、tranlation、permutationを指す。
  * Crystal Graph Convolutional Neural Network(CGCNN)を使って、結晶のdescriptorを計算する方法もある。
  * 記述子を使った学習は、いい成果出してるみたい
* matminerは[別のノートブック](https://github.com/4may/MIResearch/blob/master/matminer.ipynb)で遊んでみよう

### Algorithms

Materials Informaticsでよく使われるアルゴリズムをご紹介。

**Kernel Ridge Regression(KRR)**

非線形データを使った連続値の予測に使える幅広く使える。KRRの良いところは、ハイパーパラメータの数が少ないところ。せいぜい三つ。

$L_2$正則項をたすことで、回帰係数(重み)が0にならないようにする。

* kernel手法
* カーネル係数gamma
* 正則係数alpha

**SVM**

予測精度はKRRとそんなに変わらない。SVMの方がよりスパース性が高い。ただ、ハイパーパラメータの数が多いのでチューニングが大変。

**Gaussian Process Regression(GPR)**

訓練データがガウス過程(複数のガウス分布)により生成されていると仮定する。

GPRのメリットは以下の通り。

* 分散を見積もることが出来る点
* ハイパーパラメータがGradient Discentで計算できる点

COMBOがある。

なお、KRR、SVM、GPRのいずれも特徴量の選択が必須だったが、なかなか適切な特徴量を選択するのは難しい。そこで使われるのが、LASSOだ。

**LASSO**

RidgeRegressionとは対照的に、LASSOはできるだけ回帰係数を0にする。つまり、有用な特徴量とそうでない特徴量を明確に分けることが出来るため、しばしば特徴量の選択に使われる。

ただし、LASSOは計算コストが高く、特徴量の数が$10^9$個以上になると使えない。

そこで、[SISSO(sure independence screening and sparsifying operator)](https://arxiv.org/pdf/1710.03319.pdf)と呼ばれる手法が登場した。

**SISSO**

SISSOはLASSOと比べて、より高次元の特徴量空間に対しても使うことが出来る。

**PCA**

特徴量をそのまま使うより、特徴量の数よりも少ない数の固有ベクトルを考えた方が、次元数も少なく出来るしモデルの特性もより正確に表現できるし一石二鳥。

PCAは、入力の特徴量から新しい特徴量を生み出しているから、特徴選択(feature selection)ではなく、特徴抽出(feature extraction)と考えるべき。

**Extremly Randomized Trees(ERT)**

決定木単体だと過学習になりやすいので、複数の決定木を組み合わせるERTやRFがよく使われる。ERTのメリットは以下の通り。

* 計算が速い
* ハイパーパラメータの数が少ない
* でかいデータにも使える
* 特徴量ごとの重要度から、重要な特徴量がわかる

また、ERTはRFよりも同等かそれ以上の性能を発揮する。特に、過学習になりやすい問題に対して強みを持つ。

RFでは何らかの指標をもとに決定木を育てたが、ERTはランダムに決定木を育てる。

**NN**

予測精度の高いモデルが欲しいときは、CNNやRNNなど諸々のNNが使われる。

**Variational Auto Encodrer(VAE)**

Autoencoderの亜種。解くべきタスクはGANと同じ。ある決められた確率分布にしたがって潜在変数(encoderとdecoderの中間変数)を生成する。

**GAN**

最近はMIの分野でも使われ始めた。

アルゴリズムの選定は問題に寄り切り。

**最初は単純な方法から試そう**、例えば、KRRやERTなど。**パラメータチューニングや複雑な手法を試すのは後回し**

### 新しい材料(構造)の発見

今日に至るまで、組成(composition)から構造(structure)を予測するのは不可能とされてきた。機械学習はこの難題に挑戦する。

※未知の材料の発見とは、未知の構造を発見することと解釈できる。

機械学習による材料の発見は、二通りのパターンに分類できる。

* component prediction
* structure prediction

機械学習の用語で考えると、前者は二値分類、後者は多値分類。前者は、あるcompositionがある**単一**の構造に含まれる確率を予測する。つまり、含まれるか含まれないかの二択で考えられる。それに対して、後者は入力のcompositionから複数の構造を考える。

後者の場合、計算量がきついのでSISSOやPCA、クラスタリングで特徴量を減らす。

### 材料特性の予測

機械学習を使うことで、様々な材料特性を予測することが出来る。

* band gap
* bulk and shear moduli(応力歪みの関係性) : 
* topological states(位相特性)
* superconductivity(超電導)

**band gap**

LED、太陽光発電パネル、トランジスタなど電気に関係する材料の開発をする際に、調べる。最もよく予測対象となる特性。band gapを予測する論文は多い。

**bulk and shear moduli**

他の特性とも相関があるため、関連する論文は多い。

* bonding strength(吸着の強さ)
* thermal conductivity(熱伝導性)
* charge carrier mobility(電荷キャリア移動度)
* hardness(丈夫さ)

従来は、人間が手で実験データを集めるのは大変なので、DFTを使ってシミュレーションすることが多かったが、計算コストが高いためシミュレーションに時間がかかっていた。

機械学習で効率的に計算できないか模索中。

**superconductivity**

機械学習によって、ある材料が超電導になる温度(臨界温度)を求める。

### Adaptive Design Process と Active learning

これまで述べた研究では、原子の構造や物理的特性、安定性など様々な事柄の予測を試みてきた。これらの目的は全て、材料開発のスピードを上げることである。特に、手作業による**実験**をいかに減らせるかがポイント。

実験の試み方として、adaptive design process(適応的デザイン)というものがある。

**適応的デザイン**

統計的に効果が認められるまで実験を繰り返す方法のこと。

これら実験を加速させるには、不確かさ(uncertainty)の予測が不可欠だ。

不確かさの予測には、Gaussian Processが特に注目を集めている。また、SVRや決定木、ぶーすてぃんぐなども使われる。

## 議論

**interpretability(解釈性)**

機械学習アルゴリズムはブラックボックス。何が内部で起きているのかわからない。

動作原理を理解していないと、予期せぬ結果を招いてしまう(耳が痛い)

そのため、

Interpretabilityの定義には諸説あるが、ここでは二つの事柄に分けて考えよう。

**Transparency**

transparencyは、さらに３つの事柄に分けて考えられる。

* Simulatability
* Decomposability
* Algorithmic transparency

Simulatability

* モデルの計算を理解して、再計算することが出来る能力
* LASSOなどの次元削減方法や、決定木などはこの能力を備えている。
* 材料開発の手法(DFTやMonte calro)はこの性質を持ち合わせていない。

Decomposability

* モデルをどれだけ部品化して考えられるか。入力、パラメータ、計算、出力、・・・

Algorithmic transparency

* アルゴリズムの動作原理をわかっているか

**Post hoc explanations**

モデルから追加の情報を得られるかどうか。例えば、決定木は重要度という付加情報を持っているので、Post hoc explanationsである。

DNNは、ブラックボックスの典型例である。そのため、動作原理の解析がいくつも行われている。例えば、CNNの動作原理を視覚化して理解しようとする試みがある。NNのモデル自体はわかりづらいかもしれないが、そのモデルの入出力を観察することがモデルへの理解を助けることになる！

Post hoc explanations大事！

## 結論

MIの先端技術まとめ

* crystal graph convotional networks
* surrogate-based optimization(active learning)
  * 数値シミュレーションを機械学習の手法で近似する方法
* component prediction
* bayesian optimization
* GAN/VAEs

今後の方向性は二つある。

* より優れた機械学習アルゴリズムを作る。
  * そのために、ベンチマークとなるデータの整備は必須
* 機械学習モデルの利便性を高める。
  * 材料開発向けの汎用的なフレームワークがない
  * ある特定の問題を解くための俺俺モデルの作成に時間を割くことになる
  * matminerは良いフレームワークと思う

* 最近の論文@JAP
  * ベイズ最適化多いね
  * [Machine-learning-assisted thin-film growth: Bayesian optimization in molecular beam epitaxy of SrRuO3 thin films](https://arxiv.org/pdf/1908.00739.pdf)
  * [Materials informatics based on evolutionary algorithms: Application to search for superconducting hydrogen compounds](https://arxiv.org/pdf/1908.00746.pdf)
  * [Synergy of Binary Substitutions for Improving the Cycle Performance in LiNiO2 Revealed by Ab Initio Materials Informatics](https://arxiv.org/pdf/2001.02876.pdf)